## Importing the necessary libraries

In [13]:
import os
import cv2
import numpy as np
from PIL import Image 
import Augmentor
from Augmentor import Pipeline
import torch
import torch.nn as nn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from skimage import color
import matplotlib.pyplot as plt
import glob
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, precision_recall_fscore_support, accuracy_score, recall_score, f1_score, roc_auc_score, multilabel_confusion_matrix
from sklearn.preprocessing import label_binarize
import pandas as pd
import math
from sklearn import preprocessing
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset

## Loading the Augmented training dataset

In [7]:
######################## Generate Scale Space using Gaussian Smoothing by varying the standard deviation ################
def MultiScale(image , sigma):
    kernel_size = 3
    blurred_image = cv2.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    return blurred_image

In [10]:
SIZE = 224


train_images_S1 = []
train_images_S2 = []
train_images_S3 =[]
train_labels = []

for directory_path in glob.glob("OCTDL/Augment/train/*"):
    label = directory_path.split("/")[-1]
    

    for img_path in glob.glob(os.path.join(directory_path , "*.jpg")):
         gray_img = cv2.imread(img_path , cv2.IMREAD_COLOR)
         gray_img = cv2.resize(gray_img , (SIZE , SIZE))
         img = gray_img             
         img_S1 = MultiScale(img , 1)
         img_S2 = MultiScale(img , 2)
         img_S3 = MultiScale(img , 3)
         train_images_S1.append(img_S1)
         train_images_S2.append(img_S2)
         train_images_S3.append(img_S3)
         train_labels.append(label)

    for img_path in glob.glob(os.path.join(directory_path , "*.png")):
         gray_img = cv2.imread(img_path , cv2.IMREAD_COLOR)
         gray_img = cv2.resize(gray_img , (SIZE , SIZE))
         img = gray_img                     # np.repeat(gray_img[:, :, np.newaxis], 3, axis=2)
         img_S1 = MultiScale(img , 1)
         img_S2 = MultiScale(img , 2)
         img_S3 = MultiScale(img , 3)
         train_images_S1.append(img_S1)
         train_images_S2.append(img_S2)
         train_images_S3.append(img_S3)
         train_labels.append(label)

## Loding the test dataset

In [8]:
test_images_S1 = []
test_images_S2 = []
test_images_S3 = []
test_labels = []

for directory_path in glob.glob("OCTDL/val/*"):
    label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path , "*.jpg")):
         gray_img = cv2.imread(img_path , cv2.IMREAD_COLOR)
         gray_img = cv2.resize(gray_img , (SIZE , SIZE))
         img = gray_img                   
         img_S1 = MultiScale(img , 1)
         img_S2 = MultiScale(img , 2)
         img_S3 = MultiScale(img , 3)
         test_images_S1.append(img_S1)
         test_images_S2.append(img_S2)
         test_images_S3.append(img_S3)
         test_labels.append(label)   
        
    for img_path in glob.glob(os.path.join(directory_path , "*.png")):
         gray_img = cv2.imread(img_path , cv2.IMREAD_COLOR)
         gray_img = cv2.resize(gray_img , (SIZE , SIZE))
         img = gray_img                    
         img_S1 = MultiScale(img , 1)
         img_S2 = MultiScale(img , 2)
         img_S3 = MultiScale(img , 3)
         test_images_S1.append(img_S1)
         test_images_S2.append(img_S2)
         test_images_S3.append(img_S3)
         test_labels.append(label)    

In [11]:
print(len(train_images_S1))
print(len(train_images_S2))
print(len(train_images_S3))

print(len(test_images_S1))
print(len(test_images_S2))
print(len(test_images_S3))

8400
8400
8400
417
417
417


## Normalizing the dataset

In [14]:
train_images_S1 = np.array(train_images_S1) / 255.0
train_images_S2 = np.array(train_images_S2) / 255.0
train_images_S3 = np.array(train_images_S3) / 255.0
train_labels = np.array(train_labels)

test_images_S1 = np.array(test_images_S1) / 255.0
test_images_S2 = np.array(test_images_S2) / 255.0
test_images_S3 = np.array(test_images_S3) / 255.0
test_labels = np.array(test_labels)

## Label Encoder

In [15]:
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)

le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

X_train_S1 , Y_train , X_test_S1 , Y_test = train_images_S1 , train_labels_encoded , test_images_S1 , test_labels_encoded
X_train_S2 , X_test_S2 = train_images_S2 , test_images_S2
X_train_S3 , X_test_S3 = train_images_S3 , test_images_S3

## Converting the dataset into tesor and loader

In [17]:
y_train = torch.tensor(Y_train, dtype=torch.long)
y_test = torch.tensor(Y_test, dtype=torch.long)

X_train = torch.tensor(X_train_S3, dtype=torch.float32).permute(0, 3, 1, 2)  
X_test = torch.tensor(X_test_S3, dtype=torch.float32).permute(0, 3, 1, 2)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

num_classes = len(torch.unique(y_train))

In [18]:
print(num_classes)

7


## Defining the structure of the EfficientNet-B0 model 

In [19]:
Effi_model = models.efficientnet_b0(weights=True)  # train from scratch
Effi_model.classifier[1] = nn.Linear(Effi_model.classifier[1].in_features, num_classes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Effi_model = Effi_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(Effi_model.parameters(), lr=0.0001)
scheduler = CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-6)

/home/user/anaconda3/envs/Pragya/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Training the model

In [20]:
best_val_acc = 0.0
patience, patience_counter = 20, 0

for epoch in range(1):
    Effi_model.train()
    running_loss, correct, total = 0.0, 0, 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        outputs = Effi_model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X.size(0)
        _, predicted = torch.max(outputs, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    train_loss = running_loss / total
    train_acc = correct / total

    Effi_model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            outputs = Effi_model(X)
            loss = criterion(outputs, y)
            val_loss += loss.item() * X.size(0)
            _, predicted = torch.max(outputs, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    val_loss /= total
    val_acc = correct / total

    scheduler.step()

    print(f"Epoch {epoch+1}/200 | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_counter = 0
        torch.save(Effi_model.state_dict(), "Effi_S3_Weight.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered!")
            break

Epoch 1/200 | Train Loss: 0.7356 Acc: 0.7727 | Val Loss: 0.2521 Acc: 0.9185


## Loading the learned models for extracting features

In [21]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        self.features = model.features   
        self.avgpool = model.avgpool     

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1) 
        return x

In [ ]:
#### For Expert u1 ####
Effi_model_S1 = models.efficientnet_b0(weights=None) 
Effi_model_S1.classifier[1] = nn.Linear(Effi_model_S1.classifier[1].in_features, num_classes)

Effi_model_S1.load_state_dict(torch.load("Effi_S1_Weight.pth", map_location="cpu"))
Effi_model_S1.eval()
feature_extractor_model_S1 = FeatureExtractor(Effi_model_S1).to("cuda:0" if torch.cuda.is_available() else "cpu")

#### For Expert u2 ####
Effi_model_S2 = models.efficientnet_b0(weights=None)  # train from scratch or load pretrained
Effi_model_S2.classifier[1] = nn.Linear(Effi_model_S2.classifier[1].in_features, num_classes)

Effi_model_S2.load_state_dict(torch.load("Effi_S2_Weight.pth", map_location="cpu"))
Effi_model_S2.eval()
feature_extractor_model_S2 = FeatureExtractor(Effi_model_S2).to("cuda:0" if torch.cuda.is_available() else "cpu")

#### For Expert u3 ####
Effi_model_S3 = models.efficientnet_b0(weights=None)  # train from scratch or load pretrained
Effi_model_S3.classifier[1] = nn.Linear(Effi_model_S3.classifier[1].in_features, num_classes)

# Load trained weights
Effi_model_S3.load_state_dict(torch.load("Effi_S3_Weight.pth", map_location="cpu"))
Effi_model_S3.eval()
feature_extractor_model_S3 = FeatureExtractor(Effi_model_S3).to("cuda:0" if torch.cuda.is_available() else "cpu")

## Changing the shape of extracted feature for Evidential MAGDM

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
X_train_S11 = torch.tensor(X_train_S1, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_train_S1 shape:", X_train_S11.shape)

X_train_S22 = torch.tensor(X_train_S2, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_train_S2 shape:", X_train_S22.shape)

X_train_S33 = torch.tensor(X_train_S3, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_train_S3 shape:", X_train_S33.shape)

In [ ]:
feature_extractor_model_S1.eval()
feature_extractor_model_S1.eval()
with torch.no_grad():
    train_features_S1 = feature_extractor_model_S1(X_train_S11)

train_features_S1 = train_features_S1.cpu().numpy()
print(train_features_S1.shape) 

In [ ]:
feature_extractor_model_S2.eval()
feature_extractor_model_S2.eval()
with torch.no_grad():
    train_features_S2 = feature_extractor_model_S2(X_train_S22)

train_features_S2 = train_features_S2.cpu().numpy()
print(train_features_S2.shape)  

In [ ]:
feature_extractor_model_S3.eval()
feature_extractor_model_S3.eval()
with torch.no_grad():
    train_features_S3 = feature_extractor_model_S3(X_train_S11)

train_features_S3 = train_features_S3.cpu().numpy()
print(train_features_S3.shape)  

In [ ]:
train_Features_S1 = train_features_S1
train_Features_S2 = train_features_S2
train_Features_S3 = train_features_S3

## Defining Membership Function

In [ ]:
def membership(feature , partition):
    
    FeatureMembership = []
    b = np.max(feature)
    a = np.min(feature)
    beta = (b-a)/(partition-1)
    
    for i in range(partition):
         FeaturePart = []
         if(i==0):
             for j in range(len(feature)):
                 if(feature[j]>=a and feature[j]<=b):
                     FeaturePart.append(1-(feature[j]-a)/(b-a))
                 else:
                     FeaturePart.append(0)
             FeatureMembership.append(FeaturePart)        
         elif(i==partition-1):
             for j in range(len(feature)):
                 if(feature[j]>=a and feature[j]<=b):
                     FeaturePart.append(1-(b-feature[j])/(b-a))
                 else:
                     FeaturePart.append(0)
             FeatureMembership.append(FeaturePart)        
        
         else:
             for j in range(len(feature)):
                 if(feature[j]>=a and feature[j]<=a+i*beta):
                     FeaturePart.append(1-(a+i*beta-feature[j])/(i*beta))
                 elif(feature[j]>=a+i*beta and feature[j]<=b):
                     FeaturePart.append(1-((feature[j]-a-i*beta)/(b-a-i*beta)))
                 else:
                     FeaturePart.append(0)
             FeatureMembership.append(FeaturePart)       
    return FeatureMembership

In [ ]:
Num_Alternatives , Num_Features = train_Features_S1.shape
print(Num_Alternatives , Num_Features)

## Converting the extracted features to MAGDM problem

In [ ]:
FeaturesSet = np.hstack((train_Features_S1 , train_Features_S2 , train_Features_S3)) 
print(FeaturesSet.shape)

_ , Num_FeaturesSet = FeaturesSet.shape

## Calcultaing Ordered Weighted Beleif and Plausability Measure

In [ ]:
########################## Calculating Belief and Plausibility of each alternative corresponding to the attribute ########################
Row , Column = Num_Alternatives , Num_Features
k = 0
Partition = 5
Array = np.zeros((Num_Alternatives , Column))
Belief = np.zeros((Num_Alternatives , Num_FeaturesSet) , dtype = np.float32)
Belief_Expert_List = [] ## It is denoating the Belief corresponding to each expert in the list formt containg the corresponding array [[455*5] , [455*5] , ... , [455*5]]
weights = np.array([0.833, 0.1392, 0.0233, 0.0039, 0.0006])

############################################### Evaluating Belief #########################################################################

for i in range(Num_FeaturesSet):  ## It will calculate the memebrship for each features in the FeatreSet
    if(~(FeaturesSet[:,i] == 0).all()):
          A = (np.array(membership(FeaturesSet[ : , i] , Partition))).T  ## It will return a list of list [[],[],[],[],[]]
          Column_Sum = A.sum(axis = 0)
          Column_Sum[Column_Sum == 0] = 1
          
          A_Normalized = A/Column_Sum   ## Normalizing the membership coloumnwise to convert it into BPA
          A_Belief = np.sum(A_Normalized * weights , axis = 1)  ## Evaluating Belief with respect to one feature and giving weightage to that 

        
          # Reshape to column vector
          A_Belief = A_Belief[: , np.newaxis]
          Belief[: , i] = A_Belief.flatten()  ## Saving the belief value for each feature in a new array say Belief
    if(i == 0 or i % Column != 0):
          Array[: , k] = Belief[: , i]
          if(i == Num_FeaturesSet - 1):
              Belief_Expert_List.append(Array)
    elif(i % Column == 0):
        Belief_Expert_List.append(Array)
        Array = np.zeros((Num_Alternatives , Column))
        k = 0
        Array[: , k] = Belief[: , i]
        #print(i)
    k = k+1
    #print(i,k)
        
#print(Belief.shape)
print(len(Belief_Expert_List))

In [ ]:
#################################################### Evaluating Plausibility #################################################################

Belief_SumAll_Experts = np.sum(Belief_Expert_List , axis = 0)
Correct_Sum = Belief_SumAll_Experts
Correct_Sum[Correct_Sum == 0] = 1

#print(Belief_Expert_List[0][:,0])
#print(Belief_SumAll_Experts)

Pl_List = []  ## It is denoating the Plausibility corresponding to each expert in the list formt containg the corresponding array [[455*5] , [455*5] , ... , [455*5]]

for i in range(len(Belief_Expert_List)): 
    Exceptional_Array = np.zeros((Num_Alternatives , Column))
    for j in range(len(Belief_Expert_List)):
        if(j != i):
            Exceptional_Array += Belief_Expert_List[j]
    Pl_List.append(1 - (Exceptional_Array / Correct_Sum))        
#print(Pl_List)    
Plausibility = np.concatenate(Pl_List , axis = 1)
print(Plausibility.shape)  
#print(Pl_List)

## Computing WPBL

In [ ]:
############################## Evaluating the WPBl with respect to each expert corresponding to the features ################
PBl_List = []

PBl_neumerator = (np.array(Belief_Expert_List) + np.array(Pl_List)).tolist()  ## It is the list of array containg belief+plausiblity with respect to the features
PBl_denominator1 = [np.sum(i , axis = 1) for i in Belief_Expert_List]
PBl_denominator2 = [np.sum(j , axis = 1) for j in Pl_List]
PBl_denominator =  [arr1 + arr2 for arr1, arr2 in zip(PBl_denominator1 , PBl_denominator2)] 

print(len(PBl_neumerator))
print(len(PBl_denominator))

for array1 , array2 in zip(PBl_neumerator , PBl_denominator):
    # Check for zero values in array2
    if np.any(array2 == 0):
        
        result = np.zeros((Row , Column))
    else:
        
        result = array1 / array2[:, np.newaxis] 
    print(result)    
    PBl_List.append(result)
    
    
#print((PBl_List))    
PBl = np.concatenate(PBl_List, axis=1)

## Computing the Ordered Weighted Belief Divregence Measure

In [ ]:
########### Calcutaing the divergence between each expert for each alternative by considering all the alternatives #########
Num_Experts = len(Belief_Expert_List)
Num_Div_Pairs = ((Num_Experts - 1) * Num_Experts) // 2
Divergence_Array = np.zeros((Row , Num_Div_Pairs)) ## The second place is denoating the total number of pairs for evaluating the divergence between the experts 
K = 0

def Div(Array1 , Array2):
    Array = np.zeros((Num_Alternatives , 1))
    Row , Column = Array1.shape
    for i in range(Row):
        Sum = 0
        for j in range(Column):
            if(Array1[i][j] != 0 and Array2[i][j] != 0): 
               Sum += (Array1[i][j] * np.log2((2 * Array1[i][j]) / (Array1[i][j] + Array2[i][j]))) + (Array2[i][j] * np.log2((2 * Array2[i][j]) / (Array1[i][j] + Array2[i][j])))
            elif(Array1[i][j] == 0 and Array2[i][j] != 0):
               Sum += (Array2[i][j] * np.log2((2 * Array2[i][j]) / (Array1[i][j] + Array2[i][j])))
            elif(Array1[i][j] != 0 and Array2[i][j] == 0):
               Sum += (Array1[i][j] * np.log2((2 * Array1[i][j]) / (Array1[i][j] + Array2[i][j])))
        Array[i][0] = 0.5 * Sum
    return Array    
        
for i in range(len(PBl_List)):
    Array1 = PBl_List[i]
    for j in range(i+1 , len(PBl_List)):
        Array2 = PBl_List[j]
        Result = Div(Array1 , Array2)
        Divergence_Array[ : , K] = Result.flatten()
        K = K+1

ColumnAvg_Divergence = np.mean(Divergence_Array , axis = 0)
print(ColumnAvg_Divergence)

## Computig Diveregence Measure Matrix

In [ ]:
########################### Evaluating the weights for the fusion using the divergence matrix ##############################
Divergence_Matrix = np.zeros((Num_Experts , Num_Experts))
H = 0
l = 0
for i in range(Num_Experts):
    for j in range(l , Num_Experts):
        if(i == j):
            Divergence_Matrix[i][j] = 0
        else:
            Divergence_Matrix[i][j] = ColumnAvg_Divergence[H]
            H += 1
    l += 1       
               
print(Divergence_Matrix)
Final_Divergence_Matrix = (Divergence_Matrix + Divergence_Matrix.T)
print(Final_Divergence_Matrix)

Avg_Divergence_Matrix = np.mean(Final_Divergence_Matrix , axis =0)
print(Avg_Divergence_Matrix)
Support = 1/Avg_Divergence_Matrix

Weight_Fusion = Support/np.sum(Support , axis = 0) ## The final weight for fusing the feaures for group decision making
print(Weight_Fusion)

## Fusing the attribute infromation using conseusnus evidential weights

In [ ]:
###################################################### Final Fused Features for training #################################################################
Final_Fused_Features_train = np.zeros((Num_Alternatives , Num_Features)) 
Final_Fused_Features_train += (train_Features_S1 * Weight_Fusion[0]) + (train_Features_S2 * Weight_Fusion[1]) + (train_Features_S3 * Weight_Fusion[2])

## Defining Random Forest Classifier (RFC)

In [ ]:
RandomForest = RandomForestClassifier(n_estimators = 1000 , max_features = 25 , random_state = 42)
RandomForest.fit(Final_Fused_Features_train , Y_train)

## Passing the final fused infromation to RFC 

In [ ]:
PredictionRF_train = RandomForest.predict(Final_Fused_Features_train)
print("Train Accuracy = ", accuracy_score(Y_train , PredictionRF_train))

## Extracting the features for test dataset for validation

In [ ]:
X_test_S11 = torch.tensor(X_test_S1, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_test_S1 shape:", X_test_S11.shape)

X_test_S22 = torch.tensor(X_test_S2, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_test_S2 shape:", X_test_S22.shape)

X_test_S33 = torch.tensor(X_test_S3, dtype=torch.float32).permute(0, 3, 1, 2).to(device)
print("Raw X_test_S3 shape:", X_test_S33.shape)

In [ ]:
with torch.no_grad():
    test_features_S1 = feature_extractor_model_S1(X_test_S11)

test_features_S1 = test_features_S1.cpu().numpy()
print(test_features_S1.shape)  # should be (8400, 1280)

with torch.no_grad():
    test_features_S2 = feature_extractor_model_S2(X_test_S22)

test_features_S2 = test_features_S2.cpu().numpy()
print(test_features_S2.shape)  # should be (8400, 1280)

with torch.no_grad():
    test_features_S3 = feature_extractor_model_S3(X_test_S33)

test_features_S3 = test_features_S3.cpu().numpy()
print(test_features_S3.shape)  # should be (8400, 1280)

## Fusing the extracted features of Test Dataset

In [ ]:
###################################################### Final Fused Features for training #################################################################
test_Features_S1 = test_features_S1
test_Features_S2 = test_features_S2
test_Features_S3 = test_features_S3
Num_Alternatives_test , Num_Features_test = test_Features_S1.shape

Final_Fused_Features_test = np.zeros((Num_Alternatives_test , Num_Features_test)) 

Final_Fused_Features_test += (test_Features_S1 * Weight_Fusion[0]) + (test_Features_S2 * Weight_Fusion[1]) + (test_Features_S3 * Weight_Fusion[2])

## Using RFC trained model for fused of feature of test dataset for validation

In [ ]:
PredictionRF_test = RandomForest.predict(Final_Fused_Features_test)
print("Test Accuracy = ", accuracy_score(Y_test , PredictionRF_test))

## Computing evaluation metrics

In [ ]:
accuracy = accuracy_score(Y_test , PredictionRF_test)
cm = multilabel_confusion_matrix(Y_test, PredictionRF_test)

sensitivity = []
specificity = []
precision =[]
for i in range(len(cm)):
    tp = cm[i][1][1]  # True positives
    fn = cm[i][1][0]  # False negatives
    tn = cm[i][0][0]  # True negatives
    fp = cm[i][0][1]  # False positives
    sensitivity.append(tp / (tp + fn))
    specificity.append(tn / (tn + fp))
    precision.append(tp / (tp + fp))


f1 = f1_score(Y_test , PredictionRF_test , average = 'weighted')

auc = roc_auc_score(Y_test , RandomForest.predict_proba(Final_Fused_Features_test) , multi_class = 'ovr', average = 'macro')


print("Accuracy:", accuracy)
print("Sensitivity (Recall) for each class:", np.mean(sensitivity))
print("Specificity for each class:", np.mean(specificity))
print("F1-score:", f1)
print("AUC for each class:", auc)
print("Precision: " , np.mean(precision))

overall_auc = roc_auc_score(Y_test , RandomForest.predict_proba(Final_Fused_Features_test) , multi_class = 'ovr', average = 'macro')

print("Overall AUC:", overall_auc)